In [1]:
from safe_control_gym.envs.manipulators.manipulator import BaseManipulator
import numpy as np 
import random 
from time import sleep
from safe_control_gym.envs.benchmark_env import Cost, Task
from gym import spaces
from stable_baselines3 import PPO, DDPG, A2C

pybullet build time: Mar 29 2022 22:56:14


In [2]:
urdf_path = "../safe_control_gym/envs/manipulators/assets/franka_panda/panda.urdf"
control_mode = "torque"
target_space = "joint"

controlled_joint_indices = [6]
observed_link_indices = [7]
observed_link_state_keys = ["position"]
goal = [{
    "position": [None for i in range(13)]
}]

# orientation_goal = [0.603, 0.3687, -0.3697, 0.6026]
# position_goal = [0.03692, 0, 0.973]
position_goal = [-0.127, 0.0, 1.012]
# position_goal = [0.0, 0, 1]
goal[0]["position"][observed_link_indices[0]] = position_goal
goal_type = "point"

env = BaseManipulator(
    urdf_path,
    control_mode,
    target_space,
    controlled_joint_indices = controlled_joint_indices,
    observed_link_indices = observed_link_indices, 
    observed_link_state_keys = observed_link_state_keys,
    goal = goal,
    goal_type = goal_type,
    cost = Cost.RL_REWARD
)

argv[0]=
Version = 4.1 Metal - 76.3
Vendor = Apple
Renderer = Apple M1 Pro
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [5]:
action_space = [-4, -2, -3, 3, 2, 4]
# action_space = [-400]

for i in range(50):
#     action_index = env.n_joints-2
    action_index = 6
    # action_list = np.zeros(env.n_joints)
    action_list = [random.choice(action_space)]    
    
    # action_list = [200]
    obs,reward, done,info = env.step(action_list)
    # if i%5== 0 :
    print(action_list)
    print(f"obs: {obs}") # link position
    print(f"reward: {reward}")
  #   print(f"angular velocity: {obs}") # velocity
                
    sleep(0.05)

[2]
obs: (0.08799999952316284, 3.329974913434536e-13, 1.0329999923706055)
reward: -111.07999189410134
[-3]
obs: (0.08800004422664642, -1.245352763135088e-07, 1.0329999923706055)
reward: -111.08016113252822
[-3]
obs: (0.087978295981884, -3.838818827262003e-07, 1.031003713607788)
reward: -109.06239347155483
[3]
obs: (0.08779814839363098, -6.511243100248976e-07, 1.027016043663025)
reward: -104.89484318096592
[2]
obs: (0.08779588341712952, -9.233827569232744e-08, 1.027016043663025)
reward: -104.89201941843012
[3]
obs: (0.08779597282409668, -2.149775042425972e-07, 1.027016043663025)
reward: -104.89223146462584
[-3]
obs: (0.08790911734104156, -6.468247448765396e-08, 1.029009222984314)
reward: -106.99840500783003
[-3]
obs: (0.08790996670722961, -3.282507350377273e-07, 1.029008150100708)
reward: -106.99844505867267
[2]
obs: (0.08779759705066681, -5.850599222867459e-07, 1.0270140171051025)
reward: -104.89219921569165
[4]
obs: (0.08779644221067429, -3.037956730622682e-07, 1.0270140171051025)
rew

In [9]:
env.reset()

(0.08799999952316284, 3.329974913434536e-13, 1.0329999923706055)

Exploring link states

In [3]:
link_states =  env._pb_client.getLinkState(env.robot,
                                            linkIndex=7,
                                            computeLinkVelocity=True)


In [4]:
link_states

((0.088, -4.504557837847756e-13, 0.953),
 (1.0, 0.0, 0.0, 4.896583138958022e-12),
 (0.0, 0.0, 0.08),
 (0.0, 0.0, 0.0, 1.0),
 (0.08799999952316284, 3.329974913434536e-13, 1.0329999923706055),
 (1.0, 0.0, 0.0, 4.896583138958022e-12),
 (0.0, -0.0, 0.0),
 (0.0, -0.0, 0.0))

In [25]:
# only extract world positions
link_states_dict = {
    "position": link_states[4],
    "orientation": link_states[5],
    "linear_vel": link_states[6],
    "angular_vel": link_states[7]
}

In [26]:
link_states_dict

{'position': (0.0010245442390441895,
  -1.3434328138828278e-06,
  1.0330008268356323),
 'orientation': (0.6968757510185242,
  -0.11944480240345001,
  0.025003790855407715,
  0.7067332863807678),
 'linear_vel': (0.16140478353054072,
  -0.021992152304630256,
  -0.7820516002624263),
 'angular_vel': (2.675834999501902, 19.791254360890452, -0.004319621550829485)}

## RL

In [5]:
from gym import spaces 
# env.observation_space = spaces.Box(-np.inf, np.inf, (14,), dtype=np.float32)

# orientation use quaternion so 4 dim 

# action_space = [spaces.Discrete(1) for i in range(env.n_joints-1)]
# action_space.append(spaces.Box(-400, 400, shape=(1,), dtype=np.float32))

# env.action_space = spaces.Tuple(
#     action_space
# )

# dim dict
dim_dict = {
    "position":3 + 3,
    "orientation" : 4+4
}
env.action_space = spaces.Box(-4.5, 4.5, (len(env.controlled_joint_indices),), dtype=np.float32)

# TODO expand to multiple state keys 
env.observation_space = spaces.Box(-np.inf, np.inf, (dim_dict[observed_link_state_keys[0]], ), dtype=np.float32)

# env.observation_space = spaces.Dict({
#       'position': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#       'orientation': spaces.Box(-np.inf, np.inf, (env.n_joints, 4), dtype=np.float32),
#       'linear_vel': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#       'angular_vel': spaces.Box(-np.inf, np.inf, (env.n_joints, 3), dtype=np.float32),
#     })


In [7]:

ppo_model = PPO('MlpPolicy', env, verbose=1)

# use this to stop if too many timesteps cannot converge to done 
for i in range(10):
    ppo_model.learn(total_timesteps=10000)




Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 412      |
|    ep_rew_mean     | -7.9e+04 |
| time/              |          |
|    fps             | 47       |
|    iterations      | 1        |
|    time_elapsed    | 43       |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1.31e+03     |
|    ep_rew_mean          | -2.87e+05    |
| time/                   |              |
|    fps                  | 47           |
|    iterations           | 2            |
|    time_elapsed         | 86           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 6.363407e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    en

In [8]:
ppo_model.save("manipulator_100000")

In [10]:
obs = env.reset()
for i in range(500):
    action, _state = ppo_model.predict(obs, deterministic=False)
    # print(_state)
    obs, reward, done, info = env.step(action)
    applied_action = env._action_mapping_torque(action)
    
    if i%5==0:
        print(f"Iteration:{i}")
        print(f"action: {action}")
        print(f"goal: {goal[0]['position'][7]}")
        print(f"state: {[round(ob,3) for ob in obs]}")
        print(f"reward: {reward}\n")
    
    if done: 
        print("done")
        break
    
    sleep(0.05)

Iteration:0
action: [1.0969899]
goal: [-0.127, 0.0, 1.012]
state: [0.088, 0.0, 1.033, -0.127, 0.0, 1.012]
reward: -235.9999918941013

Iteration:5
action: [0.79837924]
goal: [-0.127, 0.0, 1.012]
state: [0.083, 0.0, 1.062, -0.127, 0.0, 1.012]
reward: -260.30347805373367

Iteration:10
action: [0.39074004]
goal: [-0.127, 0.0, 1.012]
state: [0.031, 0.0, 1.116, -0.127, 0.0, 1.012]
reward: -262.0205112741496

Iteration:15
action: [-0.17638242]
goal: [-0.127, 0.0, 1.012]
state: [0.005, 0.0, 1.121, -0.127, 0.0, 1.012]
reward: -241.0641707067407

Iteration:20
action: [-0.15105997]
goal: [-0.127, 0.0, 1.012]
state: [0.011, 0.0, 1.12, -0.127, 0.0, 1.012]
reward: -246.31752239678463

Iteration:25
action: [-0.0033571]
goal: [-0.127, 0.0, 1.012]
state: [0.04, 0.0, 1.112, -0.127, 0.0, 1.012]
reward: -266.9568698386138

Iteration:30
action: [1.367567]
goal: [-0.127, 0.0, 1.012]
state: [0.064, 0.0, 1.097, -0.127, 0.0, 1.012]
reward: -276.13823216801393

Iteration:35
action: [-0.28238034]
goal: [-0.127, 